In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matminer.data_retrieval.retrieve_Citrine import CitrineDataRetrieval
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers import composition as cf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from pymatgen import Composition
from scipy.stats import norm
from collections import Counter
from pprint import pprint
import pandas as pd
import numpy as np
import statistics
import os


/Users/malcolmdavidson/anaconda3/envs/tecca/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [2]:
# Load Data
# Compute features
# Run CV
# Report

## Configure

In [3]:
load_path = os.path.join('data','training_data.csv')
api_key = os.environ.get('CITRINATION_API_KEY')
site = 'https://citrination.com'
config_path = os.path.join('config','config.json')
np.random.seed(8)

## Compute Classes

In [4]:
# There will be 2^11 possible class labels
from itertools import product
num_class = list(product([0.0,1.0],repeat=11))

In [5]:
len(num_class)

2048

## Load data

In [6]:
data = pd.read_csv(load_path)
pprint(f"Loaded {len(data.index)} records.")

'Loaded 2572 records.'


In [7]:
data['formula'] = data['formulaA'] + data['formulaB']

In [8]:
def get_compostion(c):
    """Attempt to parse composition, return None if failed"""
    
    try:
        return Composition(c)
    except:
        return None

In [9]:
data = data.sample(1000)

In [10]:
o_feat = data

In [11]:
n_feat = StrToComposition().featurize_dataframe(data, col_id='formula')

In [12]:
o_feat.equals(n_feat.drop('composition', axis=1))

True

In [13]:
data['composition'] = data['formula'].apply(get_compostion)

In [14]:
data = data[['composition','stabilityVec']]

In [15]:
def check_nobility(row):
    comp = row['composition']
    return comp.contains_element_type('noble_gas')

In [16]:
data['noble'] = data.apply(check_nobility, axis=1)
data = data[data['noble'] == False]

In [17]:
data.reset_index(drop=True, inplace=True)

## Compute features using matminer

In [20]:
f =  MultipleFeaturizer([cf.Stoichiometry(), cf.ElementProperty.from_preset("magpie"),
                         cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])

In [21]:
%%capture
X = np.array(f.featurize_many(data['composition']))

We see that our featurized input does have a non-numeric value. This very likely due to magpie not being able to compute features for noble gases. Lets check if any row containing a noble gas is stable. If we don't see any signal we will drop those rows and re-featurize.

## Run cross validation

In [22]:
model = LogisticRegression()

In [23]:
model.classes = num_class

In [24]:
y = data['stabilityVec'].values

In [28]:
y_resid = []
y_uncer = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=8)


In [29]:
model.fit(X_train, y_train)

/Users/malcolmdavidson/anaconda3/envs/tecca/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

/Users/malcolmdavidson/anaconda3/envs/tecca/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [30]:
yf_pred = model.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score

In [32]:
print(accuracy_score(y_test, yf_pred))

0.38636363636363635
